In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8686.08it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4822.29it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2777924.27it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8979.64it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4812.69it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 89687.26it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8269.58it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4815.67it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 75361.75it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8641.43it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4839.20it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 87378.96it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.ConvKNRM()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = True
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
#model.backend.summary()

Parameter "name" set to ConvKNRM.


In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=5, num_neg=1, batch_size=20)
len(train_generator)

255

In [11]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
255/255 [==============================] - 33s 129ms/step - loss: 0.4178
Validation: normalized_discounted_cumulative_gain@3(0): 0.5644249595231345 - normalized_discounted_cumulative_gain@5(0): 0.6181641523866899 - mean_average_precision(0): 0.5820818624479569
Epoch 2/30
255/255 [==============================] - 27s 106ms/step - loss: 0.0799
Validation: normalized_discounted_cumulative_gain@3(0): 0.5645181071040578 - normalized_discounted_cumulative_gain@5(0): 0.6283527787570147 - mean_average_precision(0): 0.5800770146746018
Epoch 3/30
255/255 [==============================] - 27s 104ms/step - loss: 0.0372
Validation: normalized_discounted_cumulative_gain@3(0): 0.594064247710109 - normalized_discounted_cumulative_gain@5(0): 0.6443493868479856 - mean_average_precision(0): 0.5967726003485497
Epoch 4/30
255/255 [==============================] - 26s 104ms/step - loss: 0.0215
Validation: normalized_discounted_cumulative_gain@3(0): 0.5804273606919105 - normalized_discounted_cu

255/255 [==============================] - 25s 98ms/step - loss: 0.0000e+00
Validation: normalized_discounted_cumulative_gain@3(0): 0.6209981079538025 - normalized_discounted_cumulative_gain@5(0): 0.6683650496544457 - mean_average_precision(0): 0.6320168909359449
Epoch 30/30
255/255 [==============================] - 25s 96ms/step - loss: 0.0000e+00
Validation: normalized_discounted_cumulative_gain@3(0): 0.6209981079538025 - normalized_discounted_cumulative_gain@5(0): 0.6683650496544457 - mean_average_precision(0): 0.6320168909359449
